In [0]:
import gensim
print(f'gensim: {gensim.__version__}')

gensim: 3.6.0


In [0]:
!conda install -y tqdm

/bin/bash: conda: command not found


Solving environment: ...working... done

# All requested packages already installed.

In [0]:
from tqdm import tqdm
class TqdmUpTo(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None: self.total = tsize
        self.update(b * bsize - self.n)

def get_data(url, filename):
    """
    Download data if the filename does not exist already
    Uses Tqdm to show download progress
    """
    import os
    from urllib.request import urlretrieve
    
    if not os.path.exists(filename):

        dirname = os.path.dirname(filename)
        if not os.path.exists(dirname):
            os.makedirs(dirname)

        with TqdmUpTo(unit='B', unit_scale=True, miniters=1, desc=url.split('/')[-1]) as t:
            urlretrieve(url, filename, reporthook=t.update_to)

In [0]:
embedding_url = 'http://nlp.stanford.edu/data/glove.6B.zip'
get_data(embedding_url, 'data/glove.6B.zip')

glove.6B.zip: 862MB [06:32, 2.20MB/s]                          


In [0]:
# # We need to run this only once, can unzip manually unzip to the data directory too
!unzip data/glove.6B.zip 
!mv glove.6B.300d.txt data/glove.6B.300d.txt 
!mv glove.6B.200d.txt data/glove.6B.200d.txt 
!mv glove.6B.100d.txt data/glove.6B.100d.txt 
!mv glove.6B.50d.txt data/glove.6B.50d.txt

Archive:  data/glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [0]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file = 'data/glove.6B.300d.txt'

word2vec_output_file = 'data/glove.6B.300d.word2vec.txt'

In [0]:
import os
if not os.path.exists(word2vec_output_file):
    glove2word2vec(glove_input_file, word2vec_output_file)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


##KeyedVectors API

In [0]:
from gensim.models import KeyedVectors
filename = word2vec_output_file

In [0]:
%%time
# load the Stanford GloVe model from file, this is Disk I/O and can be slow
pretrained_w2v_model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)
# binary=False format for human readable text (.txt) files, and binary=True for .bin files

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


CPU times: user 1min 54s, sys: 1.49 s, total: 1min 56s
Wall time: 1min 55s


In [0]:
# calculate: (king - man) + woman = ?
result = pretrained_w2v_model.wv.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(result)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('queen', 0.6713277101516724)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
# calculate: (india - canada) +  = ?
result = pretrained_w2v_model.most_similar(positive=['quora', 'facebook'], negative=['linkedin'], topn=1)
print(result)

[('twitter', 0.37966805696487427)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
#pretrained_w2v_model.most_similar('bang;adesh')
#pretrained_w2v_model.most_similar('love')
pretrained_w2v_model.most_similar('india')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('loves', 0.6420263051986694),
 ('passion', 0.6261469125747681),
 ('loved', 0.595270574092865),
 ('romantic', 0.5921086668968201),
 ('lovers', 0.5904368162155151),
 ('lover', 0.5841912031173706),
 ('you', 0.5837852954864502),
 ('me', 0.5835772752761841),
 ('affection', 0.5818219184875488),
 ('always', 0.5701121687889099)]

**What is missing in both word2vec and GloVe?**

In [0]:
try:
    pretrained_w2v_model.wv.most_similar('nirant')
except Exception as e:
    print(e)

"word 'nirant' not in vocabulary"


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


**How to handle OOV words?**

In [0]:
ted_dataset = "https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip"
get_data(ted_dataset, "data/ted_en.zip")

ted_en-20160408.zip&filename=ted_en-20160408.zip: 16.0MB [00:08, 1.90MB/s]


In [0]:
import zipfile
import lxml.etree
with zipfile.ZipFile('data/ted_en.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))

In [0]:
#print(input_text[:5000])
print(input_text[:500])

Here are two reasons companies fail: they only do more of the same, or they only do what's new.
To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation. Both are necessary, but it can be too much of a good thing.
Consider Facit. I'm actually old enough to remember them. Facit was a fantastic company. They were born deep in the Swedish forest, and they made the best mechanical calculators in the world. Everybody used them. A


In [0]:
import re
# remove parenthesis 
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)

# store as list of sentences
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# store as list of lists of words
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

In [0]:
#print(sentences_ted[:5])
sentences_ted[:2]

[['here',
  'are',
  'two',
  'reasons',
  'companies',
  'fail',
  'they',
  'only',
  'do',
  'more',
  'of',
  'the',
  'same',
  'or',
  'they',
  'only',
  'do',
  'what',
  's',
  'new'],
 ['to',
  'me',
  'the',
  'real',
  'real',
  'solution',
  'to',
  'quality',
  'growth',
  'is',
  'figuring',
  'out',
  'the',
  'balance',
  'between',
  'two',
  'activities',
  'exploration',
  'and',
  'exploitation']]

#create a json file

In [0]:
import json
with open('ted_clean_sentences.json', 'w') as fp:
    json.dump(sentences_ted, fp)

In [0]:
with open('ted_clean_sentences.json', 'r') as fp:
    sentences_ted = json.load(fp)

In [0]:
print(sentences_ted[:2])

[['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new'], ['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']]


#Train FastText Embedddings

In [0]:
from gensim.models.fasttext import FastText

In [0]:
%%time
fasttext_ted_model = FastText(sentences_ted, size=100, window=5, min_count=5, workers=-1, sg=1)
# sg = 1 denotes skipgram, else CBOW is used

CPU times: user 9.28 s, sys: 942 ms, total: 10.2 s
Wall time: 10.2 s


In [0]:
fasttext_ted_model.wv.most_similar("india")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('indians', 0.5911639928817749),
 ('indian', 0.5406097769737244),
 ('indiana', 0.4898717999458313),
 ('indicated', 0.44004374742507935),
 ('indicate', 0.4042605757713318),
 ('internal', 0.39166826009750366),
 ('interior', 0.3871103823184967),
 ('byproducts', 0.37529298663139343),
 ('princesses', 0.37265270948410034),
 ('indications', 0.369659960269928)]

#Train word2vec Embeddings

In [0]:
from gensim.models.word2vec import Word2Vec

In [0]:
%%time
word2vec_ted_model = Word2Vec(sentences=sentences_ted, size=100, window=5, min_count=5, workers=-1, sg=1)

CPU times: user 5.59 s, sys: 36.2 ms, total: 5.62 s
Wall time: 5.63 s


In [0]:
word2vec_ted_model.wv.most_similar("india")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('straight', 0.36203503608703613),
 ('smith', 0.34995749592781067),
 ('royal', 0.3473374843597412),
 ('yale', 0.3392207622528076),
 ('weaknesses', 0.3338955044746399),
 ('biscuits', 0.3306465148925781),
 ('suicidal', 0.3214688301086426),
 ('attachments', 0.32116562128067017),
 ('sargassum', 0.3197120428085327),
 ('charts', 0.31942853331565857)]

**fastText or word2vec?**

#Document Embeddings

In [0]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import gensim
from pprint import pprint
import multiprocessing

In [0]:
import zipfile
import lxml.etree
with zipfile.ZipFile('data/ted_en.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
    
talks = doc.xpath('//content/text()')

In [0]:
def read_corpus(talks, tokens_only=False):
    for i, line in enumerate(talks):
        if tokens_only:
            yield gensim.utils.simple_preprocess(line)
        else:
            # For training data, add tags
            yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [0]:
read_corpus(talks)

<generator object read_corpus at 0x7fbc8c8dc518>

In [0]:
ted_talk_docs = list(read_corpus(talks))

In [0]:
print(ted_talk_docs[0])

TaggedDocument(['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 'new', 'to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation', 'both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'good', 'thing', 'consider', 'facit', 'actually', 'old', 'enough', 'to', 'remember', 'them', 'facit', 'was', 'fantastic', 'company', 'they', 'were', 'born', 'deep', 'in', 'the', 'swedish', 'forest', 'and', 'they', 'made', 'the', 'best', 'mechanical', 'calculators', 'in', 'the', 'world', 'everybody', 'used', 'them', 'and', 'what', 'did', 'facit', 'do', 'when', 'the', 'electronic', 'calculator', 'came', 'along', 'they', 'continued', 'doing', 'exactly', 'the', 'same', 'in', 'six', 'months', 'they', 'went', 'from', 'maximum', 'revenue', 'and', 'they', 'were', 'gone', 'gon

In [0]:
cores = multiprocessing.cpu_count()
print(cores)

2


In [0]:
model = Doc2Vec(dm=0, vector_size=100, negative=5, hs=0, min_count=2, epochs=5, workers=cores)

In [0]:
%time model.build_vocab(ted_talk_docs)

CPU times: user 8.5 s, sys: 110 ms, total: 8.61 s
Wall time: 8.63 s


In [0]:
sentence_1 = 'Modern medicine has changed the way we think about healthcare, life spans and by extension career and marriage'

In [0]:

sentence_2 = 'Modern medicine is not just a boon to the rich, making the raw chemicals behind these is also pollutes the poorest neighborhoods'

In [0]:
sentence_3 = 'Modern medicine has changed the way we think about healthcare, and increased life spans, delaying weddings'

In [0]:
model.docvecs.similarity_unseen_docs(model, sentence_1.split(), sentence_3.split())

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


-0.029946072

In [0]:
model.docvecs.similarity_unseen_docs(model, sentence_1.split(), sentence_2.split())

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


-0.2710956

In [0]:
%time model.train(ted_talk_docs, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 40.9 s, sys: 200 ms, total: 41.1 s
Wall time: 21.4 s


In [0]:
model.infer_vector(sentence_1.split())

array([ 0.05769451,  0.02035589, -0.1454503 , -0.06174285, -0.02289042,
       -0.10213989, -0.08131486, -0.09142482, -0.01179488,  0.11928166,
       -0.02452498, -0.02419745, -0.00607277, -0.1483039 ,  0.05172996,
        0.13771448,  0.03145176, -0.19651736, -0.13508199, -0.00478769,
       -0.07966698, -0.04353228,  0.06385525, -0.05305105, -0.0202617 ,
       -0.15225968,  0.003328  ,  0.0540623 , -0.15258513, -0.1081012 ,
        0.12409838, -0.05138121, -0.09012511, -0.07147333, -0.14038157,
        0.18344541, -0.09624001,  0.01520134,  0.10863149, -0.04114882,
       -0.06682971, -0.05707677,  0.17753433, -0.06419012,  0.00927438,
        0.04007374,  0.0860406 , -0.17649081, -0.05031716, -0.09726327,
       -0.20834132, -0.10361291, -0.02176658, -0.08745968, -0.01601041,
       -0.00867863, -0.10202109,  0.14325444,  0.03028674,  0.00912757,
       -0.04258632,  0.11543725,  0.03265692, -0.09924871,  0.0896194 ,
        0.05071303,  0.10801093,  0.01203658, -0.01569997,  0.01

In [0]:
model.docvecs.similarity_unseen_docs(model, sentence_1.split(), sentence_3.split())

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.90387654

In [0]:
model.docvecs.similarity_unseen_docs(model, sentence_1.split(), sentence_2.split())

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.66175693

In [0]:
model.docvecs.similarity_unseen_docs(model, sentence_2.split(), sentence_3.split())

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.7485609

# Model Assessment

In [0]:
ranks = []
for idx in range(len(ted_talk_docs)):
    inferred_vector = model.infer_vector(ted_talk_docs[idx].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(idx)
    ranks.append(rank)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
import collections
collections.Counter(ranks)  # Results vary due to random seeding + very small corpus

Counter({0: 2080, 1: 1, 3: 3, 4: 1})

In [0]:
doc_slice = ' '.join(ted_talk_docs[idx].words)[:500]
print(f'Document ({idx}): «{doc_slice}»\n')
print(f'SIMILAR/DISSIMILAR DOCS PER MODEL {model}')
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
      doc_slice = ' '.join(ted_talk_docs[sims[index][0]].words)[:500]
      print(f'{label} {sims[index]}: «{doc_slice}»\n')

Document (2084): «if you re here today and very happy that you are you ve all heard about how sustainable development will save us from ourselves however when we re not at ted we are often told that real sustainability policy agenda is just not feasible especially in large urban areas like new york city and that because most people with decision making powers in both the public and the private sector really don feel as though they re in danger the reason why here today in part is because of dog an abandoned puppy»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dbow,d100,n5,mc2,s0.001,t2)
MOST (2084, 0.9415737390518188): «if you re here today and very happy that you are you ve all heard about how sustainable development will save us from ourselves however when we re not at ted we are often told that real sustainability policy agenda is just not feasible especially in large urban areas like new york city and that because most people with decision making powers in both the public and the priv